In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from fbprophet import Prophet

In [7]:
def region_dataframe(region):
    """
    returns dataframe with the following related to specific region:
    estimated_fire_area
    mean_Precipitation
    mean_RelativeHumidity
    mean_SoilWaterContent
    mean_SolarRadiation
    mean_Temperature
    mean_Windspeed
    vegetation_index_mean
    """
    
    wildfires = pd.read_csv('Jan_09/Historical_wildfires.csv')
    wildfires['Date'] = pd.to_datetime(wildfires['Date'])
    wildfires = pd.pivot(wildfires, index = 'Date', columns = 'Region', values = ['Estimated_fire_area', 'Count'])
    columns = []
    for i,x in wildfires.columns:
        columns.append('{}_{}'.format(i,x))
    wildfires.columns = columns
    
    
    # Instantiating another dataframe with a daterange and merge to find the missing date.
    dummy = pd.DataFrame(index = pd.date_range(start = '2005-01-01', end = '2021-01-02'))


    # 2020-03-06 have missing values for all 7 states.
    wildfires = pd.merge(wildfires, dummy, how = 'outer', left_index= True, right_index=True)
    wildfires.fillna(0, inplace=True)
    print(wildfires.shape)

    df = wildfires[['Estimated_fire_area_{}'.format(region), 'Count_{}'.format(region)]]
    
    
    weather = pd.read_csv('Jan_09/HistoricalWeather.csv')
    weather.columns = ['Date', 'Region', 'Parameter', 'count', 'min', 'max', 'mean', 'variance']
    weather['Date'] = pd.to_datetime(weather['Date'])
    weather = weather.loc[weather['Region'].eq(region)]
    weather_pivot = pd.pivot(weather, index=['Date'], columns = ['Parameter', 'Region'], values = 'mean')

    columns = []
    for i,x in weather_pivot.columns:
        columns.append('mean_{}_{}'.format(i,x))
    weather_pivot.columns = columns
    weather_pivot.fillna(method = 'ffill', inplace=True)
    
    
    
    
    # Here we have monthly data that needs to be changed to daily.
    vegetation = pd.read_csv('Jan_09/VegetationIndex.csv')
    vegetation['Date'] = pd.to_datetime(vegetation['Date'])
    vegetation = vegetation.loc[vegetation['Region'].eq(region)]

    vegetation_pivot = pd.pivot(vegetation, index='Date', columns = 'Region').resample('1D').mean()
    vegetation_pivot = vegetation_pivot[['Vegetation_index_mean']]
    vegetation_pivot = vegetation_pivot[:'2020-10-01'].interpolate()
    vegetation_pivot = vegetation_pivot.unstack().reset_index()
    vegetation_pivot.columns = ['param', 'region', 'ds', 'y']
    vegetation_pivot = vegetation_pivot[['ds', 'y']]
    
    
    m = Prophet(
    changepoint_prior_scale= 30,
    holidays_prior_scale = 20,
    seasonality_prior_scale = 35,
    n_changepoints = 100,
    seasonality_mode = 'additive',
    daily_seasonality = False,
    weekly_seasonality=False,
    yearly_seasonality = False
        ).add_seasonality(name = 'monthly', period = 30.5, fourier_order=12
        ).add_seasonality(name = 'weekly', period = 7, fourier_order = 20
        ).add_seasonality(name = 'yearly', period = 365.25, fourier_order = 20
        ).add_seasonality(name = 'quarterly', period = 365.25/4, fourier_order = 5, prior_scale=15)
    
    print('fitting vegetation for {}'.format(region))
    m.fit(vegetation_pivot)
    future = m.make_future_dataframe(periods=93)
    pred = m.predict(future)
    vegetation_pivot = pred[['ds', 'yhat']]
    vegetation_pivot.rename(columns = {'ds':'Date', 'yhat':'Vegetation_index_mean_{}'.format(region)}, inplace= True)
    vegetation_pivot.set_index('Date', inplace=True)
    
#     for i in vegetation_pivot.columns:
#         vegetation_pivot[i] = vegetation_pivot[i].interpolate()
#     dummy = pd.DataFrame(index = pd.date_range(start = '2020-12-02', end = '2021-01-08'))
#     vegetation_pivot = pd.concat([vegetation_pivot, dummy], axis = 0)
#     for i in vegetation_pivot.columns:
#         vegetation_pivot[i] = vegetation_pivot[i].fillna(method='ffill')

        
        
#     columns = []
#     for i,x in vegetation_pivot.columns:
#         columns.append('{}_{}'.format(i,x))
#     vegetation_pivot.columns = columns

    df = pd.concat([df, weather_pivot, vegetation_pivot], axis = 1)

    df.reset_index(inplace=True)
    df.rename(columns ={'index':'Date'}, inplace=True)
    
    return df


In [8]:
regions = ['NSW', 'NT', 'QL', 'SA', 'TA', 'VI', 'WA']

In [9]:
for i in regions:
    region_dataframe(i).to_csv('{}_iso.csv'.format(i), index = False, header= True)

(5852, 14)


INFO:numexpr.utils:NumExpr defaulting to 4 threads.


fitting vegetation for NSW


C:\Users\albert\Anaconda3\lib\site-packages\pandas\core\frame.py:4446: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(5852, 14)
fitting vegetation for NT


...

C:\Users\albert\Anaconda3\lib\site-packages\pandas\core\frame.py:4446: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(5852, 14)
fitting vegetation for VI


C:\Users\albert\Anaconda3\lib\site-packages\pandas\core\frame.py:4446: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(5852, 14)
fitting vegetation for WA


C:\Users\albert\Anaconda3\lib\site-packages\pandas\core\frame.py:4446: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

